# Visualize results

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.2
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.9.0
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.15.1
pyscf~=2.8.0; sys.platform != 'win32'
```
</details>

## Plot histogram <a name='histogram'></a>

The `plot_histogram` function visualizes the result of sampling a quantum circuit on a QPU or simulator.

<Admonition title="Using the output from functions" type="tip">
This function returns a `matplotlib.Figure` object. When the last line of a code cell outputs these objects, Jupyter notebooks display them below the cell. If you call these functions in some other environments or in scripts, you will need to explicitly show or save the outputs.

Two options are:
- Call `.show()` on the returned object to open the image in a new window (assuming your configured matplotlib backend is interactive).
- Call `.savefig("out.png")` to save the figure to `out.png` in the current working directory. The `savefig()` method takes a path so you can adjust the location and filename where you're saving the output. For example, `plot_state_city(psi).savefig("out.png")`.
</Admonition>

For example, make a two-qubit Bell state:

In [1]:
from qiskit_aer.primitives import Sampler

from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram

sampler = Sampler(backend_options={"seed_simulator": 1234})

In [2]:
# quantum circuit to make a Bell state
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# execute the quantum circuit
quasi_dists = sampler.run(bell, shots=1000).result().quasi_dists[0]
print(quasi_dists)

{3: 0.485, 0: 0.515}


In [3]:
plot_histogram(quasi_dists)

<Image src="/images/extracted-notebook-images/visualize-results/f217705f-f930-411f-be6e-ea06eb5c877b-0.svg" alt="Output from the previous code" />

### Options when plotting a histogram

Use the following options for `plot_histogram` to adjust the output graph.

* `legend`: Provides a label for the executions. It takes a list of strings used to label each execution's results. This is mostly useful when plotting multiple execution results in the same histogram
* `sort`: Adjusts the order of the bars in the histogram. It can be set to either ascending order with `asc` or descending order with `desc`
* `number_to_keep`: Takes an integer for the number of terms to show. The rest are grouped together in a single bar called "rest"
* `color`: Adjusts the color of the bars; takes a string or a list of strings for the colors to use for the bars for each execution
* `bar_labels`: Adjusts whether labels are printed above the bars
* `figsize`: Takes a tuple of the size in inches to make the output figure

In [4]:
# Execute two-qubit Bell state again
second_quasi_dists = sampler.run(bell, shots=1000).result().quasi_dists[0]

# Plot results with custom options
plot_histogram(
    [quasi_dists, second_quasi_dists],
    legend=["first", "second"],
    sort="desc",
    figsize=(15, 12),
    color=["orange", "black"],
    bar_labels=False,
)

<Image src="/images/extracted-notebook-images/visualize-results/2c67a5f1-20c0-4d95-bfb5-37509862880f-0.svg" alt="Output from the previous code" />

## Plotting estimator results

Qiskit does not have a built-in function for plotting Estimator results, but you can use Matplotlib's [`bar` plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html) for a quick visualization.

To demonstrate, the following cell estimates the expectation values of seven different observables on a quantum state.

In [5]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator
from matplotlib import pyplot as plt

# Simple estimation experiment to create results
qc = QuantumCircuit(2)
qc.h(0)
qc.crx(1.5, 0, 1)

observables_labels = ["ZZ", "XX", "YZ", "ZY", "XY", "XZ", "ZX"]
observables = [SparsePauliOp(label) for label in observables_labels]

estimator = Estimator(backend_options={"seed_simulator": 1234})
result = estimator.run([qc] * 7, observables).result()
print(result)

# Plot using Matplotlib
plt.bar(observables_labels, result.values)

EstimatorResult(values=array([ 0.46875   ,  0.0390625 ,  0.50390625, -0.03125   , -0.70898438,
        0.00390625,  0.74609375]), metadata=[{'shots': 1024, 'variance': 0.7802734375, 'simulator_metadata': [{'time_taken': 0.082658566, 'num_bind_params': 1, 'parallel_state_update': 4, 'parallel_shots': 1, 'required_memory_mb': 1, 'input_qubit_map': [[1, 1], [0, 0]], 'method': 'statevector', 'device': 'CPU', 'num_qubits': 2, 'sample_measure_time': 0.028617122, 'active_input_qubits': [0, 1], 'num_clbits': 2, 'remapped_qubits': False, 'runtime_parameter_bind': False, 'max_memory_mb': 15990, 'noise': 'ideal', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'applied': False, 'max_fused_qubits': 5, 'threshold': 14, 'enabled': True}}]}, {'shots': 1024, 'variance': 0.99847412109375, 'simulator_metadata': [{'time_taken': 0.09200582, 'num_bind_params': 1, 'parallel_state_update': 4, 'parallel_shots': 1, 'required_memory_mb': 1, 'input_qubit_map': [[1, 1], [0, 0]], 'method'

<BarContainer object of 7 artists>

<Image src="/images/extracted-notebook-images/visualize-results/d97cd321-7acf-45a7-bfc3-823e225154c7-2.svg" alt="Output from the previous code" />

The following cell estimates the [standard error](https://en.wikipedia.org/wiki/Standard_error) from the [variance](https://en.wikipedia.org/wiki/Variance) of each result and adds them as error bars. See the [`bar` plot documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html) for a full description of the plot.

In [6]:
from math import sqrt

standard_error = [
    sqrt(exp_data["variance"]) / sqrt(exp_data["shots"])
    for exp_data in result.metadata
]

_, ax = plt.subplots()
ax.bar(observables_labels, result.values, yerr=standard_error, capsize=2)
ax.set_title("Expectation values (with standard errors)")

Text(0.5, 1.0, 'Expectation values (with standard errors)')

<Image src="/images/extracted-notebook-images/visualize-results/4b4b532c-bf79-42bf-bc6a-3877e132c89a-1.svg" alt="Output from the previous code" />